In [15]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

NUMERIC_FEATURES = [
    'p1_height',
    'p2_height',
    'p1_age',
    'p2_age',
    'p1_rating',
    'p2_rating',
    'p1_dev',
    'p2_dev',
    'p1_surface_rating',
    'p2_surface_rating',
    'p1_surface_dev',
    'p2_surface_dev',
    'p1_w',
    'p2_w',
    'p1_l',
    'p2_l',
    'p1_surface_w',
    'p2_surface_w',
    'p1_surface_l',
    'p2_surface_l',
    'p1_inactive_days',
    'p2_inactive_days',
    'p1_recent_rating',
    'p2_recent_rating'
]
CATEGORICAL_FEATURES = [
    'is_hard',
    'is_clay',
    'is_grass',
    'is_bo5',
    'p1_lefty',
    'p2_lefty',
    'p1_home',
    'p2_home'
    
]

def model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    keras.layers.InputLayer(input_shape=(28,)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.45),
    keras.layers.Dense(1, activation='sigmoid')
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam())
  return model

dataframe = pd.read_csv('../data/matches.csv')[:50000]

def get_model(dataframe):
    labels = dataframe.pop('p1_win')
    dataframe.pop('match_hash')
    dataframe.pop('tourney_name')
    dataframe.pop('tourney_date')
    dataframe.pop('p1_name')
    dataframe.pop('p2_name')

    print('dataframe shape', dataframe.shape)

    dataframe_features = dataframe.copy()

    inputs = {}

    # match column names with input objects
    for name, column in dataframe_features.items():
        dtype = tf.float32
        inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

    numeric_inputs = {name:input for name,input in inputs.items()
                    if name in NUMERIC_FEATURES}

    # normalize the numeric inputs and gather them in an array
    x = keras.layers.Concatenate()(list(numeric_inputs.values()))
    norm = keras.layers.Normalization()
    norm.adapt(np.array(dataframe[numeric_inputs.keys()]))
    all_numeric_inputs = norm(x)

    preprocessed_inputs = [all_numeric_inputs]

    for name, input in inputs.items():
        if name in NUMERIC_FEATURES:
            continue
        preprocessed_inputs.append(input)

    preprocessed_inputs_cat = keras.layers.Concatenate()(preprocessed_inputs)
    preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

    features_dict = {name: np.array(value) for name, value in dataframe_features.items()}
    f_dict = {name:values for name, values in features_dict.items()}
    preprocessing(f_dict)

    data_model = model(preprocessing, inputs)

    return data_model, features_dict, labels

data_model, features_dict, labels = get_model(dataframe)
data_model.fit(x=features_dict, y=labels, epochs=50)
data_model.save('test')


# num_rows = dataframe.shape[0]
# training_end_index = math.floor((2/3) * num_rows)
# # train_dataframe = dataframe.iloc[:training_end_index]
# # val_dataframe = dataframe.iloc[training_end_index:]
# train_dataframe = dataframe.iloc[:300]
# val_dataframe = dataframe.iloc[300:450]

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (c:\Users\andyw\OneDrive\Desktop\Projects\tennis-ml\env\lib\site-packages\tensorflow\python\eager\def_function.py)

In [ ]:
reloaded = tf.keras.models.load_model('test')
validation = pd.read_csv('../data/matches.csv')[50000:50100]
_, features_dict, labels = get_model(validation)
results = reloaded.evaluate(features_dict, labels)
print(results)
# predictions = reloaded.predict(features_dict)
# for i in range(50000, 50100):
#     pred_index = i - 50000
#     print('Prediction:', predictions[pred_index], 'Result:', labels[i])

#     pass
#     # print('prediction:', predictions[i], 'result:', labels[i])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


c:\Users\andyw\OneDrive\Desktop\Projects\tennis-ml\env\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  if update_shape and output_rank >= 3:


4/4 [==============================] - 0s 4ms/step - loss: 0.6259
0.625948429107666
